#### questao 5 considere o large movie dataset a) Remova as stopwords das resenhas do dataset.

In [2]:
import numpy as np
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import nltk

nltk.download('stopwords')
nltk.download('wordnet')


def carregar_dados_imdb():
    dados_treino = load_files("../aclImdb/train/", categories=['pos', 'neg'], encoding='utf-8', random_state=42)
    dados_teste = load_files("../aclImdb/test/", categories=['pos', 'neg'], encoding='utf-8', random_state=42)
    return dados_treino.data, dados_treino.target, dados_teste.data, dados_teste.target


def remover_stopwords(textos):
    stop_words = set(stopwords.words('english'))
    tokenizer = RegexpTokenizer(r'\w+')
    textos_filtrados = []
    for texto in textos:
        palavras = tokenizer.tokenize(texto.lower())
        filtrado = ' '.join([palavra for palavra in palavras if palavra not in stop_words])
        textos_filtrados.append(filtrado)
    return textos_filtrados





X_train, y_train, X_test, y_test = carregar_dados_imdb()
X_train_sw = remover_stopwords(X_train)

print(f" textos sem stopwords: {X_train_sw[0:20]}")



[nltk_data] Downloading package stopwords to /home/gab42/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/gab42/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


 textos sem stopwords: ['really post comments wanted make sure warn people film unfinished student film redeeming features whatsoever technical level completely amateur constant unintentional jump edits within scenes dubbing wildly etc plot completely clichéd structure laughable acting embarrassing want harsh made share student films awful reason film world innocent fans see br br safe assumption much like cast positive comments filmmakers friends family', 'let begin saying big fantasy fan however film many far fetched arguments trying support film claim dragons possibly ever existed film mentions connections different stories different countries fails investigate thoroughly could given film credibility film uses nice cgi tell us narrated fantasy story young dragon life combined popular tv show csi style flash forwards make look like something scientific definitely many cases arguments clues far fetched cases clues used show dragons possibly existed flew spit fire simply invalid see ma

##### questao 5 b) Realize o stemming das expressões contidas nas features resultantes da operação realizada no item (a).


In [3]:

def aplicar_stemming(textos):
    stemmer = PorterStemmer()
    tokenizer = RegexpTokenizer(r'\w+')
    textos_stemmed = []
    for texto in textos:
        palavras = tokenizer.tokenize(texto)
        stemmed = ' '.join([stemmer.stem(palavra) for palavra in palavras])
        textos_stemmed.append(stemmed)
    return textos_stemmed

X_train_stem = aplicar_stemming(X_train_sw)
print(f"textos com stemming {X_train_stem[0:30]}")

textos com stemming ['realli post comment want make sure warn peopl film unfinish student film redeem featur whatsoev technic level complet amateur constant unintent jump edit within scene dub wildli etc plot complet clichéd structur laughabl act embarrass want harsh made share student film aw reason film world innoc fan see br br safe assumpt much like cast posit comment filmmak friend famili', 'let begin say big fantasi fan howev film mani far fetch argument tri support film claim dragon possibl ever exist film mention connect differ stori differ countri fail investig thoroughli could given film credibl film use nice cgi tell us narrat fantasi stori young dragon life combin popular tv show csi style flash forward make look like someth scientif definit mani case argument clue far fetch case clue use show dragon possibl exist flew spit fire simpli invalid see make get cramp toe even fantasi film need degre realiti one bottom line pretenti fantasi csi documentari worth watch', 'realli b

##### questao 5 c) Realize a lemmatization das expressões contidas nas features resultantes da operação realizada no item (a)


In [4]:

def aplicar_lemmatization(textos):
    lemmatizer = WordNetLemmatizer()
    tokenizer = RegexpTokenizer(r'\w+')
    textos_lemmatizados = []
    for texto in textos:
        palavras = tokenizer.tokenize(texto)
        lemmatizado = ' '.join([lemmatizer.lemmatize(palavra) for palavra in palavras])
        textos_lemmatizados.append(lemmatizado)
    return textos_lemmatizados

X_train_lem = aplicar_lemmatization(X_train_sw)
print(f" palavras apos com lemmatizatio {X_train_stem[0:20]}")

 palavras apos com lemmatizatio ['realli post comment want make sure warn peopl film unfinish student film redeem featur whatsoev technic level complet amateur constant unintent jump edit within scene dub wildli etc plot complet clichéd structur laughabl act embarrass want harsh made share student film aw reason film world innoc fan see br br safe assumpt much like cast posit comment filmmak friend famili', 'let begin say big fantasi fan howev film mani far fetch argument tri support film claim dragon possibl ever exist film mention connect differ stori differ countri fail investig thoroughli could given film credibl film use nice cgi tell us narrat fantasi stori young dragon life combin popular tv show csi style flash forward make look like someth scientif definit mani case argument clue far fetch case clue use show dragon possibl exist flew spit fire simpli invalid see make get cramp toe even fantasi film need degre realiti one bottom line pretenti fantasi csi documentari worth watch

#### questao 6 - Considere o Large Movie Review Dataset a) Redimensione os dados do dataset pelo método term frequency–inverse document frequency (tf-idf). Apresente os resultados obtidos.


In [5]:
def redimensionar_tfidf(textos):
    vetor_config = {"min_df": 5, "stop_words": None, "ngram_range": (1, 2)}
    vetor = TfidfVectorizer(**vetor_config)
    matriz = vetor.fit_transform(textos)
    print(f"\nNúmero de documentos: {matriz.shape[0]}")
    print(f"Número de features (palavras/ngramas): {matriz.shape[1]}")
    print(f"Primeiras 10 features: {vetor.get_feature_names_out()[:10]}")
    return vetor, matriz

matriz_tfidf = redimensionar_tfidf(X_train_sw)


Número de documentos: 20476
Número de features (palavras/ngramas): 76884
Primeiras 10 features: ['00' '000' '000 00' '000 000' '000 budget' '000 feet' '000 year'
 '000 years' '007' '01']


### questao 6 b) Crie um modelo de classificação baseado em regressão logística sobre a base redimensionada no item (a) e avalie os resultados obtidos.


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

def treinar_modelo_logistico(X, y):
    parametros = {
        'C': [0.01, 0.1, 1, 10],
        'penalty': ['l2'],
        'solver': ['liblinear', 'lbfgs']
    }
    modelo = LogisticRegression(max_iter=1000, random_state=42)
    grid = GridSearchCV(modelo, parametros, cv=5, scoring='accuracy', n_jobs=-1)
    grid.fit(X, y)
    print(f"Melhores parâmetros: {grid.best_params_}")
    return grid.best_estimator_


def avaliar_modelo(modelo, X_teste, y_teste):
    y_pred = modelo.predict(X_teste)
    print("\nRelatório de Classificação:")
    print(classification_report(y_teste, y_pred))
    print(f"Acurácia: {accuracy_score(y_teste, y_pred):.4f}")


X_test_sw = remover_stopwords(X_test)
vetor, matriz_tfidf_train = redimensionar_tfidf(X_train_sw)
matriz_tfidf_test = vetor.transform(X_test_sw)

modelo = treinar_modelo_logistico(matriz_tfidf_train, y_train)
avaliar_modelo(modelo, matriz_tfidf, y_test)


Número de documentos: 20476
Número de features (palavras/ngramas): 76884
Primeiras 10 features: ['00' '000' '000 00' '000 000' '000 budget' '000 feet' '000 year'
 '000 years' '007' '01']
Melhores parâmetros: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}


ValueError: Expected 2D array, got 1D array instead:
array=[TfidfVectorizer(min_df=5, ngram_range=(1, 2))
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 2680313 stored elements and shape (20476, 76884)>].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.